# Development Notebook
This notebook is intended to test the functions to implement these into main function.

In [8]:
import openai
import os
from openai import OpenAI

In [9]:
# Read API key from the local due to the security
with open(r"C:\Users\Issei\Documents\API Tokens\ChatGPT.txt", 'r') as file:
    OPEN_AI_APIKEY = file.read().rstrip()

client = OpenAI(
  api_key=OPEN_AI_APIKEY
  )

In [16]:
def connection_test(user_input):
    completion = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages=[
            {
                'role': 'system',
                'content': 'You are test chatbot to check my API function worked or not. Generate a sentence'
            },
            {
                'role': 'user',
                'content': f"testing user input is:{user_input}"
            }
        ]
    )
    
    print(completion.choices[0].message.content)

In [17]:
#test the fuction

connection_test('Hi GPT')

Hello! How can I assist you today?


In [18]:
# Set your API key
openai.api_key = OPEN_AI_APIKEY

# List available models
models = client.models.list()

print(models)

SyncPage[Model](data=[Model(id='gpt-3.5-turbo', created=1677610602, object='model', owned_by='openai'), Model(id='gpt-4o', created=1715367049, object='model', owned_by='system'), Model(id='gpt-4o-mini', created=1721172741, object='model', owned_by='system')], object='list')


In [45]:
import datetime
def parse_schedule(user_input):
    completion = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
            {
                'role': 'system',
                'content':f"""
                Extract the event details from input and 'today' is {datetime.date.today()}
                Output Format: {{"date": "YYYY-MM-DD", "time": "HH:MM", "event": "Event"}}
                """
            },
            {
                'role': 'user',
                'content': f'{user_input}'
            }
        ],
        max_tokens=100
    )
    return completion.choices[0].message.content


In [46]:
user_input = "来週の月曜日、午後3時にミーティングを追加して"
parsed_schedule = parse_schedule(user_input)
print(parsed_schedule)

{"date": "2024-12-02", "time": "15:00", "event": "ミーティング"}


In [2]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# 認証フローを起動して認証済みトークンを生成
flow = InstalledAppFlow.from_client_secrets_file(
    r"C:\Users\Issei\Documents\API Tokens\CalenderSecretary\client_secret_630503106817-im66k9rhjgpu3us9b8dvkhsi4j9qku7d.apps.googleusercontent.com.json",
    scopes=['https://www.googleapis.com/auth/calendar']
)

creds = flow.run_local_server(port=8080)

# トークンを保存して再利用（オプション）
with open('token.json', 'w') as token:
    token.write(creds.to_json())

# Google Calendar APIクライアントを作成
service = build('calendar', 'v3', credentials=creds)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=630503106817-im66k9rhjgpu3us9b8dvkhsi4j9qku7d.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=zeRpNlRFGkSNQ5mw3lSc69tDsgzRTE&access_type=offline


In [54]:
from datetime import timedelta, datetime

def add_event(service, date, time, event_name):
    start_time = datetime.strptime(f"{date} {time}", "%Y-%m-%d %H:%M")
    end_time = start_time + timedelta(hours=1)  # デフォルトで1時間

    event = {
        'summary': event_name,
        'start': {
                'dateTime': start_time.isoformat(),
                'timeZone': 'Asia/Tokyo',
        },
        'end': {
                'dateTime': end_time.isoformat(),
                'timeZone': 'Asia/Tokyo'
        }
    }

    event = service.events().insert(calendarId='primary', body=event).execute()
    print(f"Event created: {event.get('htmlLink')}")

In [41]:
user_input = input("スケジュールを入力してください: ")


In [55]:
#parsed_schedule = eval(parsed_schedule)  # 出力が文字列の場合、辞書に変換

# 2. Google Calendarにイベントを登録
add_event(
    service,
    parsed_schedule['date'],
    parsed_schedule['time'],
    parsed_schedule['event']
)


Event created: https://www.google.com/calendar/event?eid=Y3UzZHJ0cTJlNTByaXE2YjRxcHVmdGNxaDQgbWF0Y2gubWF0c3V6b2VAbQ
